# OAT - Convert Daily Yields to Monthly Averages

This notebook reads the daily France 10-Year Yield data and converts it to monthly averages.

The output CSV will contain:
- Date: Last day of each month
- Value: Monthly average of daily yields


In [ ]:
import pandas as pd
import os
import warnings
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

warnings.filterwarnings('ignore')


In [ ]:
# Change to project root directory
# Find the project root by looking for the 'data' directory
current_dir = os.getcwd()
while not os.path.exists(os.path.join(current_dir, 'data')):
    parent_dir = os.path.dirname(current_dir)
    if parent_dir == current_dir:
        # Reached filesystem root without finding 'data' directory
        raise FileNotFoundError("Could not find project root directory (looking for 'data' folder)")
    current_dir = parent_dir

os.chdir(current_dir)
print(f"Working directory: {os.getcwd()}")

# Define input and output files
input_file = 'data/oat/FRANCE_10_YEARS_DAILY_YIELDS.csv'
output_file = 'data/oat/FRANCE_10_YEARS_MONTHLY_YIELDS.csv'

# Check that the input file exists
if not os.path.exists(input_file):
    raise FileNotFoundError(f"File not found: {input_file}")

print(f"Input file: {input_file}")
print(f"Output file: {output_file}")


In [ ]:
# Read the daily yields CSV file
print("Reading daily yields CSV file...")
df_daily = pd.read_csv(input_file)

print(f"Data shape: {df_daily.shape}")
print(f"\nColumn names:")
print(df_daily.columns.tolist())
print(f"\nFirst few rows:")
print(df_daily.head(10))
print(f"\nLast few rows:")
print(df_daily.tail(10))


In [ ]:
# Convert date column to datetime
df_daily['date'] = pd.to_datetime(df_daily['date'], format='%Y-%m-%d', errors='coerce')

# Remove rows with invalid dates
df_daily = df_daily[df_daily['date'].notna()].copy()

# Ensure value column is numeric
df_daily['value'] = pd.to_numeric(df_daily['value'], errors='coerce')

# Remove rows with invalid values
df_daily = df_daily[df_daily['value'].notna()].copy()

# Sort by date
df_daily = df_daily.sort_values(by='date').reset_index(drop=True)

print(f"Date range: {df_daily['date'].min()} to {df_daily['date'].max()}")
print(f"Total daily rows: {len(df_daily)}")
print(f"\nFirst few rows:")
print(df_daily.head(10))


In [ ]:
# Create year-month column for grouping
df_daily['year_month'] = df_daily['date'].dt.to_period('M')

# Group by year-month and calculate monthly average
df_monthly = df_daily.groupby('year_month')['value'].mean().reset_index()
df_monthly.columns = ['year_month', 'value']

print(f"Monthly averages calculated: {len(df_monthly)} months")
print(f"\nFirst few rows:")
print(df_monthly.head(10))


In [ ]:
# Convert period to datetime and get the last day of each month
# Using to_timestamp with how='end' to get the last day of the month
df_monthly['date'] = df_monthly['year_month'].dt.to_timestamp(how='end')

# Select only date and value columns
df_output = df_monthly[['date', 'value']].copy()

print(f"Final monthly data: {len(df_output)} rows")
print(f"Date range: {df_output['date'].min()} to {df_output['date'].max()}")
print(f"\nFirst few rows:")
print(df_output.head(10))
print(f"\nLast few rows:")
print(df_output.tail(10))


In [ ]:
# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Format date as string (YYYY-MM-DD) for CSV output
df_to_save = df_output.copy()
df_to_save['date'] = df_to_save['date'].dt.strftime('%Y-%m-%d')

# Save to CSV (format: date,value)
df_to_save.to_csv(output_file, index=False)
print(f"File saved successfully: {output_file}")
print(f"Dimensions: {len(df_to_save)} rows, {len(df_to_save.columns)} columns")
print(f"\nFinal data summary:")
print(df_output.describe())


In [ ]:
# Create a plot of the monthly data
plt.figure(figsize=(16, 8))
plt.plot(df_output['date'], df_output['value'], linewidth=1.2, color='#2E86AB', alpha=0.8, marker='o', markersize=3)
plt.title('OAT - France 10-Year Yield (Monthly Averages) - Historical Trend', 
          fontsize=14, fontweight='bold')
plt.xlabel('Data', fontsize=12)
plt.ylabel('Rendimento (%)', fontsize=12)
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()

# Format x-axis dates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.YearLocator(2))  # Show every 2 years
plt.xticks(rotation=45)

# Save plot
plot_file = os.path.join(os.path.dirname(output_file), 'FRANCE_10_YEARS_MONTHLY_YIELDS_plot.png')
plt.savefig(plot_file, dpi=150, bbox_inches='tight')
print(f"Plot saved to: {plot_file}")

plt.show()
